# Create time-calibrated time series of land-cover probabilities

### Create time-calibrated land-cover probabilities using multi-year training data
* Many classes, for testing only woodland class
* subset of southern Chiquitania with training data fram that area
* Goal: extract a stack of annual land-cover probabilites

#### Long-term goal:
* smooth these probabilities using land-trendr

In [1]:
import ee  # the Earth Engine Python API library
ee.Initialize()
import ipyleaflet  # an interactive mapping "widget" for the notebook
from baumiTools import baumiEE, baumiVT
import json
import ogr, osr
import os
#import pandas as pd

## Load the shapefiles
1. The point shapefile, in which we have the training data
2. The landsat WRS-path/row data. These we need to do the sampling better.
3. Tiles, for which we want to download the classification probabilities

#### Also set:
* variables which we need for the identification in the shapefiles
* output-Folder and root-folder

In [2]:
tileShp = bt.baumiVT.CopyToMem("D:/GEE_Landsat-compositing/CHACO_testData/CHACO_extent_plusX_tiles_Chaco_test.shp")
outputRoot = "Z:/CHACO/_LANDSAT/panChromatic_Metrics/"
years = [1985,2000]#range(2000, 2018+1, 1)
tilevar = "Id"
maxTasks = 6

## Load the layers, get a test tile

In [47]:
testID = 410
lyr = tileShp.GetLayer()
feat = lyr.GetNextFeature()
while feat:
    if feat.GetField(tilevar) == testID:
        geom = feat.GetGeometryRef()
        geom_cl = geom.Clone()
        lyr.ResetReading()
        break
    else:
        feat = lyr.GetNextFeature()
geom_json = json.loads(geom_cl.ExportToJson())
geom_coord = geom_json['coordinates']
geom_EE = ee.Geometry.Polygon(coords=geom_coord)
#geom_EE.getInfo()



## Define Image Collection in CCDC
* Find information from: https://gee-tutorials.readthedocs.io/en/latest/ccdc.html
* get visual inspireation on the segments from: https://glance.earthengine.app/view/visualizeccd

In [45]:

ccdc_0020 = ee.ImageCollection("projects/CCDC/v2")
ccdc = ccdc_0020.filterMetadata('system:index', 'starts_with', 'v5_')\
        .filterBounds(geom_EE)\
        .mosaic()\
        .select(['SWIR1_coefs'])

#print(ccdc.bandNames().getInfo())
#print(" number of image: ",ccdc.size().getInfo())
#img = ccdc.toBands()
print(ccdc.getInfo())

{'type': 'Image', 'bands': [{'id': 'SWIR1_coefs', 'data_type': {'type': 'PixelType', 'precision': 'double', 'dimensions': 2}, 'crs': 'EPSG:4326', 'crs_transform': [1.0, 0.0, 0.0, 0.0, 1.0, 0.0]}]}


In [48]:
path = ccdc.getDownloadUrl({
    'scale': 30,
    'crs': 'EPSG:4326',
    'region': geom_EE
})

EEException: Malformed JSON: 'ee.Geometry({
  "type": "Polygon",
  "coordinates": [
    [
      [
        -64.07985176168364,
        -25.06667583984569
      ],
      [
        -64.07985176168364,
        -24.13332165084217
      ],
      [
        -63.19983145315058,
        -24.13332165084217
      ],
      [
        -63.19983145315058,
        -25.06667583984569
      ],
      [
        -64.07985176168364,
        -25.06667583984569
      ]
    ]
  ],
  "evenOdd": true
})'.

In [43]:
import numpy as np
def LatLonImg(img):
    img = img.addBands(ee.Image.pixelLonLat())
 
    img = img.reduceRegion(reducer=ee.Reducer.toList(),\
                                        geometry=geom_EE,\
                                        maxPixels=1e13,\
                                        scale=100);
 
    data = np.array((ee.Array(img.get("SWIR1_coefs")).getInfo()))
    lats = np.array((ee.Array(img.get("latitude")).getInfo()))
    lons = np.array((ee.Array(img.get("longitude")).getInfo()))
    return lats, lons, data
lat, lon, data = LatLonImg(ccdc)

EEException: Array: Unrecognized type, may only use numbers or lists, found type Array.